In [ ]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
DATA_PATH_ORIG = r'../data/capture20110812.pcap.netflow.labeled.csv'
DATA_PATH_NEW = r'../data/capture20110812.pcap.netflow.labeled_processed.csv'

In [ ]:
# read the file and erase the '->'
# it takes quite a few minutes :(

# text = open(DATA_PATH, "r")
# text = ''.join([i for i in text]) \
#     .replace("->", "")
# x = open(DATA_PATH_NEW,"w")
# x.writelines(text)
# x.close()

In [ ]:
column_names = [
    '_',
    '_2',
    'duration',
    'protocol',
    'src_addr',
    'dst_addr',
    'flags',
    'tos',
    'packets',
    'bytes',
    'flows',
    'label'
]

In [ ]:
df = pd.read_csv(
    DATA_PATH_NEW,
    delimiter='\s+',
    skiprows=[0],
    header=None, 
    names=column_names,
    index_col=False,
    parse_dates={"start_date": [0,1]}
)
df.head()

In [90]:
working_df = df[:]
# print(working_df.groupby('src_addr').src_addr.count().sort_values(ascending=False)[:10])
# print(working_df.groupby('dst_addr').dst_addr.count().sort_values(ascending=False)[:10])

In [91]:
working_df['label'].unique()

array(['Background', 'LEGITIMATE', 'Botnet'], dtype=object)

In [ ]:
host_ip = "147.32.80.9:53" # this we believe is the host we are monitoring

In [92]:
# let's build an iterable with all the other IPs this guy connects with
connections = working_df[(working_df['src_addr'] == host_ip) | (working_df['dst_addr'] == host_ip)][['src_addr', 'dst_addr']]

other_ips = connections.apply(lambda row: row['dst_addr'] if row['src_addr'] == host_ip else row['src_addr'], axis=1)

counts = other_ips.value_counts().sort_values(ascending=False)
counts.head(10)

147.32.87.23:32865     2210
147.32.86.98:32797     1954
147.32.85.101:1027     1828
147.32.87.27:33003     1400
147.32.84.15:32846     1340
147.32.84.15:32849     1202
147.32.87.23:32868      498
147.32.86.193:3862      472
147.32.85.109:32831     348
147.32.84.59:49158      280
dtype: int64

In [11]:
# sns.distplot(counts, hist=False)

# Reservoir
https://stackoverflow.com/questions/12732982/design-a-storage-algorithm/12733515#12733515     
https://stackoverflow.com/questions/2612648/reservoir-sampling       
http://web.archive.org/web/20141026071430/http://propersubset.com:80/2010/04/choosing-random-elements.html

### assignment says: "Use a range of reservoir sizes"

In [232]:
# one pass
reservoir_size = 50000
ips = other_ips
stored_ips = []
           
for N, ip in enumerate(ips):
    if N < reservoir_size:
        # as long as we have space, the probability of storing is 100%
        stored_ips.append(ip)
    else:
        # when N = i+1 we have i IPs stored (i > reservoir_size) 
        # In such a case, we want probability i/i+1 of storing the new IP, substituting one of the stored ones.        
        m = random.randint(0,N)
        if m < reservoir_size:
            stored_ips[m] = ip

50000


In [208]:
print(len(counts))
print(reservoir_size)

482299
50000


3

In [103]:
counts.head(10)

147.32.87.23:32865     2210
147.32.86.98:32797     1954
147.32.85.101:1027     1828
147.32.87.27:33003     1400
147.32.84.15:32846     1340
147.32.84.15:32849     1202
147.32.87.23:32868      498
147.32.86.193:3862      472
147.32.85.109:32831     348
147.32.84.59:49158      280
dtype: int64

In [230]:
pd.Series(stored_ips).value_counts().sort_values(ascending=False).head(10)

147.32.86.98:32797     21
147.32.84.15:32846     19
147.32.87.23:32865     18
147.32.87.27:33003     16
147.32.85.101:1027     16
147.32.84.15:32849     12
147.32.86.20:64744      6
147.32.86.20:65249      6
147.32.84.138:54505     6
147.32.84.138:39368     6
dtype: int64